!pip install pymystem3 selenium html2text requests beautifulsoup4 atlas json2xml  

Don't forget to install **[Chrome Driver](https://sites.google.com/a/chromium.org/chromedriver/downloads)**

In [1]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 1. Парсинг URL адресов с новостями (Selenium)

In [3]:
#Download driver using the link at the top of the notebook
chrome_driver_path = "chromedriver.exe"
links_filename = "links.txt"

In [4]:
categories = ["politics", "starlife", "economics", "business", "tech", "incidents"]

In [5]:
def parse_category_rambler(category, pages, output):
    prefs = {"profile.managed_default_content_settings.images": 2,
             "profile.default_content_settings.cookies": 2}
    chrome_options = Options()
    chrome_options.add_extension('adblock.crx')
    chrome_options.add_experimental_option("prefs",prefs)
    chrome_options.add_argument('--disable-application-cache')
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
    driver.implicitly_wait(10)
    driver.close()
    print("Current session is {}".format(driver.session_id))
    driver.switch_to.window(driver.window_handles[0])
    driver.get('chrome-extension://gighmmpiobklfepjocnamgkkbiglidom/options.html')
    driver.find_element_by_id('acceptable_ads').click()#adblock customization
    driver.get('https://news.rambler.ru/politics/latest/?page=1')
    driver.find_element_by_class_name('j-footer__switch').click()

    for page in tqdm.tqdm(range(1, pages + 1),position=1):
        driver.get('https://news.rambler.ru/' + category + '/latest/?page=' + str(page))
        #for element in tqdm.tqdm(driver.find_elements_by_class_name('article-card'),position=0):
        for element in tqdm.tqdm(driver.find_elements_by_xpath("//article"),position=0):
            with open(output,'a') as file:
                file.write(json.dumps({
                    'link' : element.find_element_by_tag_name('a').get_attribute('href'),
                    'category': category
                },ensure_ascii=False) + '\n')
    driver.quit()

In [6]:
for cat in categories[:1]:
    parse_category_rambler(cat, 3, links_filename)

Current session is c6a2d68fe54ef05ef8f2805654334d29



100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 78.46it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 86.94it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 81.73it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.89s/it]


## 2. Парсинг Rambler новостей (BeautifulSoup + html2text)

In [7]:
posts = []
with open(links_filename) as file:
    for line in file:
        posts.append(json.loads(line))

In [8]:
for element in tqdm.tqdm(posts):
    soup = BeautifulSoup(requests.get(element['link']).text, "lxml")
    text = ''
    tags = []
    for paragraph in soup.findAll('p',attrs={'class': ''}):
        text += html2text(paragraph.text).replace('\n', ' ') + ' '

    with open('rambler_news.json','a') as file:
        file.write(json.dumps({
            'link' : element['link'],
            'category': element['category'],
            'text': text,
        },ensure_ascii=False) + '\n')

100%|████████████████████████████████████████████████████████████████████████████████| 360/360 [04:00<00:00,  1.50it/s]


In [9]:
title

'США накануне переговоров назвали РФ главной угрозой безопасности'

## 3. Обзор полученных данных

In [10]:
news = []
with open('rambler_news.json') as file:
    for line in file:
        news.append(json.loads(line))

In [11]:
data = pd.DataFrame(news)

In [12]:
data.head()

link  category  \
0  https://news.rambler.ru/politics/47918915-poso...  politics   
1  https://news.rambler.ru/politics/47917000-zaha...  politics   
2  https://news.rambler.ru/politics/47916969-ne-s...  politics   
3  https://news.rambler.ru/politics/47916724-puti...  politics   
4  https://news.rambler.ru/politics/47916719-puti...  politics   

                                               title  \
0      Посол США в России оценил переговоры в Женеве   
1  Захарова: РФ ожидает от США оперативного ответ...   
2  «Не страшна зима»: на Украине зачитали рэп в ч...   
3  Путин обсудил с Совбезом работу в приграничных...   
4            Путин назначил нового посла РФ в Тунисе   

                                                text  \
0  Посол США в России Джон Салливан прокомментиро...   
1  Москва ожидает от Вашингтона оперативного отве...   
2  Украинский музыкант Владимир Завгородний посвя...   
3  Президент России Владимир Путин на заседании с...   
4  Президент России Владимир Путин освободил Серг...   

                                                tags  
0  [Политика, Видео, Джон Салливан, НАТО, ООН, ОБ...  
1  [Политика, Видео, Мария Захарова, Администраци...  
2  [Политика, Владимир Завгородний, Ирина Венедик...  
3  [Политика, Видео, Владимир Путин, Совет безопа...  
4  [Политика, Александр Бикантов, Александр Золот...

## 4. Конвертирование в JSON формат

In [ ]:
js = json.loads(data.to_json(force_ascii=False, orient='records'))

In [ ]:

with open('corpus.json', 'w') as file:
    for i in js:
        file.write(json.dumps(i) + '\n')


## 5. Newspaper

In [ ]:
!pip install newspaper3k

In [25]:
import json
from newspaper import Config
from newspaper import Article
from newspaper.utils import BeautifulSoup

USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10

base_url = 'https://www.bbc.com/news/health-54500673'
article = Article(base_url, config=config)
article.download()
article.parse()

print("article.title: ", article.title)

article_meta_data = article.meta_data

article_summary = {value for (key, value) in article_meta_data.items() if key == 'description'}
print("article_summary: ", article_summary)

soup = BeautifulSoup(article.html, 'html.parser')
bbc_dictionary = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

date_published = [value for (key, value) in bbc_dictionary.items() if key == 'datePublished']
print("date_published: ", date_published)

article_author = [value['name'] for (key, value) in bbc_dictionary.items() if key == 'author']
print("article_author: ", article_author)

# another method to extract the title
article_title = [value for (key, value) in bbc_dictionary.items() if key == 'headline']
print("article_title: ", article_title)

article.title:  Covid virus ‘survives for 28 days’ in lab conditions
article_summary:  {'Researchers find SARS-Cov-2 survives for longer than thought - but only under certain conditions.'}
date_published:  ['2020-10-11T20:11:33.000Z']
article_author:  ['BBC News']
article_title:  ['Covid virus ‘survives for 28 days’ in lab conditions']
